## 03/03/2019

- Train a seperate model for segments in each channel.
- Investigate the performance of many classification models.
- Investigate the effects of feature selection using recursive feature elimination.
- Features used are still normalised with respect to typical response.

In [69]:
# Import necessary modules. Set settings. Import data.
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import pywt
import math
from epdata_tools import epdata_main, get_ep_features, get_ep_feature_dict
from IPython.display import HTML

from Augmentation import data_augmentation
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_selection import SelectFromModel, RFECV
from sklearn import svm, naive_bayes, neighbors, gaussian_process
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.gaussian_process.kernels import RBF
from scipy.spatial.distance import euclidean
from statsmodels.robust import mad
from fastdtw import fastdtw
from FeatureExtraction.feature_tools import detect_peaks
from tsfresh.feature_extraction import feature_calculators
from sklearn.utils import resample

import xgboost

from IPython.display import display, clear_output
import pdb

plt.style.use('default')

X = pd.read_pickle('/Users/matthewashman/github/MasterProject2018/Data/X_all_channel_labels.pkl')

## Data Format
Each row corresponds to a recording of the patient's response to either an S1 or S2 pulse at a particular coupling interval (as identified by the 'Coupling Interval' column) in a particular channel (as identified by the 'Channel' column).

In [3]:
# Remove bad files with bad labels
X = X[~(X['Label 1']=='-1') & ~(X['Label 2']=='-1')]
X.head()

,Channel,Coupling Interval,Data,Patient,S1/S2,Type,Label 1,Label 2
0,CS1-2,340,"[-636, -617, -652, -560, -482, -415, -383, -46...",1,S2,af,0,0
1,CS1-2,340,"[-903.0, -873.0, -935.0, -941.0, -910.0, -845....",1,S1,af,NaN,NaN
2,CS1-2,340,"[-931.0, -896.0, -896.0, -906.0, -858.0, -839....",1,S1,af,NaN,NaN
3,CS3-4,340,"[472, 464, 491, 523, 553, 706, 1019, 1404, 164...",1,S2,af,0,0
4,CS3-4,340,"[298.0, 292.0, 303.0, 311.0, 299.0, 395.0, 451...",1,S1,af,NaN,NaN


Split the data into S1 and S2 DataFrames. 

In [6]:
X_S1 = X.loc[X['S1/S2']=='S1']
X_S2 = X.loc[X['S1/S2']=='S2']

## Training/Test Split
Divide the patients into training/test datasets

In [7]:
# Perform training test split on patients. i.e., 3 af_patients in the test set and 7 in training.
af_patients = X[(X['Type']=='af') & (X['S1/S2']=='S2')]['Patient'].unique()
at_patients = X[(X['Type']=='at') & (X['S1/S2']=='S2')]['Patient'].unique()
avnrt_patients = X[(X['Type']=='avnrt') & (X['S1/S2']=='S2')]['Patient'].unique()
avrt_patients = X[(X['Type']=='avrt') & (X['S1/S2']=='S2')]['Patient'].unique()
ep_patients = X[(X['Type']=='ep') & (X['S1/S2']=='S2')]['Patient'].unique()

random.shuffle(af_patients); random.shuffle(at_patients); random.shuffle(avrt_patients); random.shuffle(avnrt_patients); random.shuffle(ep_patients)

train_af_patients, test_af_patients = train_test_split(af_patients, test_size=0.3)
train_at_patients, test_at_patients = train_test_split(at_patients, test_size=0.3)
train_avnrt_patients, test_avnrt_patients = train_test_split(avnrt_patients, test_size=0.3)
train_avrt_patients, test_avrt_patients = train_test_split(avrt_patients, test_size=0.3)
train_ep_patients, test_ep_patients = train_test_split(ep_patients, test_size=0.3)

# Store trining and test patients in dictionaries
training_patients = {}
training_patients['af'] = train_af_patients
training_patients['at'] = train_at_patients
training_patients['avnrt'] = train_avnrt_patients
training_patients['avrt'] = train_avrt_patients
training_patients['ep'] = train_ep_patients

test_patients = {}
test_patients['af'] = test_af_patients
test_patients['at'] = test_at_patients
test_patients['avnrt'] = test_avnrt_patients
test_patients['avrt'] = test_avrt_patients
test_patients['ep'] = test_ep_patients

In [8]:
X_train = pd.concat([X_S2[(X_S2['Type']=='af') & ([(x in train_af_patients) for x in X_S2['Patient'].values])],
                     X_S2[(X_S2['Type']=='at') & ([(x in train_af_patients) for x in X_S2['Patient'].values])],
                     X_S2[(X_S2['Type']=='avnrt') & ([(x in train_af_patients) for x in X_S2['Patient'].values])],
                     X_S2[(X_S2['Type']=='avrt') & ([(x in train_af_patients) for x in X_S2['Patient'].values])],
                     X_S2[(X_S2['Type']=='ep') & ([(x in train_af_patients) for x in X_S2['Patient'].values])]])

X_test = pd.concat([X_S2[(X_S2['Type']=='af') & ([(x in test_af_patients) for x in X_S2['Patient'].values])],
                     X_S2[(X_S2['Type']=='at') & ([(x in test_af_patients) for x in X_S2['Patient'].values])],
                     X_S2[(X_S2['Type']=='avnrt') & ([(x in test_af_patients) for x in X_S2['Patient'].values])],
                     X_S2[(X_S2['Type']=='avrt') & ([(x in test_af_patients) for x in X_S2['Patient'].values])],
                     X_S2[(X_S2['Type']=='ep') & ([(x in test_af_patients) for x in X_S2['Patient'].values])]])

# Training the Models
Choice of either training a single model to predict fractionation given an input segment from any channel, or training three seperate models to predict fractionation in for segments of a particular channel. Let's start with a single model.

In [9]:
# Proportion of responses that are fractionated
prop_fractionated = np.sum(np.float64(X_train['Label 1'].values))/X_train.shape[0]
prop_fractionated

0.0779816513761468

## Data Augmentation
As the dataset is very asymetric (many more non-fractionated examples), here I augmented more fractionated examples as described in my technical milestone report. I have found that this improves classification performance very significantly - especially in reducing the number of false negatives (i.e. fractionated responses being predicted as non-fractionated).

In [63]:
# %matplotlib qt 
# Create new augmented data for each S2 row of X_compact
X_train['Augmented'] = 0
# Use for storing augmented_rows in the form of dicts
augmented_list = []
for _, row in X_train[X_train['Label 1']=='1'].iterrows():
    
    augmented_data = data_augmentation.augment_fractionation(row['Data'], 7, False)
    for data in augmented_data:
        augmented_row = {}
        augmented_row['Data'] = data
        augmented_row['Channel'] = row['Channel']
        augmented_row['Coupling Interval'] = row['Coupling Interval']
        augmented_row['Label 1'] = row['Label 1']
        augmented_row['Label 2'] = row['Label 2']
        augmented_row['Patient'] = row['Patient']
        augmented_row['S1/S2'] = row['S1/S2']
        augmented_row['Type'] = row['Type']
        augmented_row['Augmented'] = 1
        augmented_list.append(augmented_row)
    

augmented_data = pd.DataFrame(augmented_list)

In [50]:
X_train_aug = pd.concat([X_train, augmented_data], ignore_index=True)

In [73]:
X_train_1 = X_train[X_train['Label 1']=='1']
X_train_0 = X_train[X_train['Label 1']=='0']
X_train_1_resampled = resample(X_train_1, 
                               replace=True,     # sample with replacement
                               n_samples=X_train_0.shape[0],    # to match majority class
                               random_state=123) # reproducible results
X_train_resampled = pd.concat([X_train_1_resampled, X_train_0], ignore_index=True)

In [74]:
X_train_resampled

,Channel,Coupling Interval,Data,Patient,S1/S2,Type,Label 1,Label 2,Augmented
0,CS5-6,290,"[-1257, -10464, -17582, -15528, -1689, 11195, ...",1,S2,ep,1,1,0
1,CS5-6,260,"[-451, -908, -1594, -2297, -3218, -4019, -3170...",1,S2,af,1,2,0
2,CS3-4,260,"[-263, -470, -617, -598, -921, -1036, -983, -8...",8,S2,af,1,1,0
3,CS3-4,270,"[-239, -174, -192, -283, -267, -223, -229, -30...",9,S2,af,1,1,0
4,CS5-6,260,"[-1945, -1770, -1681, -1625, -1630, -1606, -14...",9,S2,af,1,1,0
5,CS5-6,260,"[-806, -587, -414, -170, 5, 220, 153, 176, -21...",5,S2,af,1,2,0
6,CS5-6,250,"[-1263, -1085, -852, -768, -657, -416, -324, -...",5,S2,af,1,1,0
7,CS3-4,270,"[80, 208, 704, 1104, 1728, 2352, 2560, 1712, -...",4,S2,avrt,1,1,0
8,CS1-2,300,"[1013, 877, 620, 121, -569, -1556, -2997, -431...",8,S2,af,1,1,0
9,CS1-2,270,"[-148, -151, -86, -81, -120, -180, -76, -72, -...",9,S2,af,1,1,0


## Feature Extraction
Here the feature vectors are extracted for each row S2 response (including the augmented responses). A reference feature vector is subtracted (as described - the first S1 response of the patient (i.e. in the file with the largest S1/S2 coupling interval) is extracted and the feature vector calculated. This is subtracted from all feature vectors corresponding to all other S2 responses for that patient).

In [51]:
X_train_aug_feature_list = []
X_train_feature_list = []
X_test_feature_list = []
for i, row in X_train_aug.iterrows():
    clear_output(wait=True)
    display('Extracting Training Features: ' + str(round(100*i/X_train_aug.index[-1],3)) + '%')
    
    # Get typical response for this patient and channel
    typical_response = X_S1[(X_S1['Type']==row['Type']) & 
                           (X_S1['Patient']==row['Patient']) &
                           (X_S1['Channel']==row['Channel'])
                           ].sort_values(by=['Coupling Interval'], ascending=False).iloc[0]
    
    typical_feature_dict = get_good_feature_dict(typical_response['Data'])
    feature_dict = get_good_feature_dict(row['Data'])
    
    # Normalise by subtracting 'typical' feature values
    for k, v in feature_dict.items():
        feature_dict[k] = v - typical_feature_dict[k]
        
    # Fill in the other column values
    for col, value in row.iteritems():
        feature_dict[col] = value
        
    X_train_aug_feature_list.append(feature_dict)
    
for i, row in X_train.iterrows():
    clear_output(wait=True)
    display('Extracting Training Features: ' + str(round(100*i/X_train.index[-1],3)) + '%')
    
    # Get typical response for this patient and channel
    typical_response = X_S1[(X_S1['Type']==row['Type']) & 
                           (X_S1['Patient']==row['Patient']) &
                           (X_S1['Channel']==row['Channel'])
                           ].sort_values(by=['Coupling Interval'], ascending=False).iloc[0]
    
    typical_feature_dict = get_good_feature_dict(typical_response['Data'])
    feature_dict = get_good_feature_dict(row['Data'])
    
    # Normalise by subtracting 'typical' feature values
    for k, v in feature_dict.items():
        feature_dict[k] = v - typical_feature_dict[k]
        
    # Fill in the other column values
    for col, value in row.iteritems():
        feature_dict[col] = value
        
    X_train.append(feature_dict)
    
for i, row in X_test.iterrows():
    clear_output(wait=True)
    display('Extracting Test Features: ' + str(round(100*i/X_test.index[-1],3)) + '%')
    
    # Get typical response for this patient and channel
    typical_response = X_S1[(X_S1['Type']==row['Type']) & 
                           (X_S1['Patient']==row['Patient']) &
                           (X_S1['Channel']==row['Channel'])
                           ].sort_values(by=['Coupling Interval'], ascending=False).iloc[0]
    typical_feature_dict = get_good_feature_dict(typical_response['Data'])
    feature_dict = get_good_feature_dict(row['Data'])
    
    # Normalise by subtracting 'typical' feature values
    for k, v in feature_dict.items():
        feature_dict[k] = v - typical_feature_dict[k]
        
    # Fill in the other column values
    for col, value in row.iteritems():
        feature_dict[col] = value
        
    X_test_feature_list.append(feature_dict)

'Extracting Test Features: 100.0%'

In [52]:
X_train_features = pd.DataFrame(X_train_feature_list)
X_test_features = pd.DataFrame(X_test_feature_list)

In [58]:
x_training_cs12 = X_train_features[X_train_features['Channel']=='CS1-2'].drop(['Channel', 'Coupling Interval', 'Data', 'Label 1', 'Label 2', 'Patient', 'Type', 'S1/S2'], axis=1)
y_training_cs12 = X_train_features[X_train_features['Channel']=='CS1-2']['Label 1']
info_training_cs12 = X_train_features[X_train_features['Channel']=='CS1-2'][['Channel', 'Coupling Interval', 'Data', 'Label 1', 'Label 2', 'Patient', 'Type', 'S1/S2']]
x_test_cs12 = X_test_features[X_train_features['Channel']=='CS1-2'].drop(['Channel', 'Coupling Interval', 'Data', 'Label 1', 'Label 2', 'Patient', 'Type', 'S1/S2'], axis=1)
y_test_cs12 = X_test_features[X_train_features['Channel']=='CS1-2']['Label 1']
info_test_cs12 = X_test_features[X_train_features['Channel']=='CS1-2'][['Channel', 'Coupling Interval', 'Data', 'Label 1', 'Label 2', 'Patient', 'Type', 'S1/S2']]

x_training_cs34 = X_train_features[X_train_features['Channel']=='CS3-4'].drop(['Channel', 'Coupling Interval', 'Data', 'Label 1', 'Label 2', 'Patient', 'Type', 'S1/S2'], axis=1)
y_training_cs34 = X_train_features[X_train_features['Channel']=='CS3-4']['Label 1']
info_training_cs34 = X_train_features[X_train_features['Channel']=='CS3-4'][['Channel', 'Coupling Interval', 'Data', 'Label 1', 'Label 2', 'Patient', 'Type', 'S1/S2']]
x_test_cs34 = X_test_features[X_train_features['Channel']=='CS3-4'].drop(['Channel', 'Coupling Interval', 'Data', 'Label 1', 'Label 2', 'Patient', 'Type', 'S1/S2'], axis=1)
y_test_cs34 = X_test_features[X_train_features['Channel']=='CS3-4']['Label 1']
info_test_cs34 = X_test_features[X_train_features['Channel']=='CS3-4'][['Channel', 'Coupling Interval', 'Data', 'Label 1', 'Label 2', 'Patient', 'Type', 'S1/S2']]

x_training_cs56 = X_train_features[X_train_features['Channel']=='CS5-6'].drop(['Channel', 'Coupling Interval', 'Data', 'Label 1', 'Label 2', 'Patient', 'Type', 'S1/S2'], axis=1)
y_training_cs56 = X_train_features[X_train_features['Channel']=='CS5-6']['Label 1']
info_training_cs56 = X_train_features[X_train_features['Channel']=='CS5-6'][['Channel', 'Coupling Interval', 'Data', 'Label 1', 'Label 2', 'Patient', 'Type', 'S1/S2']]
x_test_cs56 = X_test_features[X_train_features['Channel']=='CS5-6'].drop(['Channel', 'Coupling Interval', 'Data', 'Label 1', 'Label 2', 'Patient', 'Type', 'S1/S2'], axis=1)
y_test_cs56 = X_test_features[X_train_features['Channel']=='CS5-6']['Label 1']
info_test_cs56 = X_test_features[X_train_features['Channel']=='CS5-6'][['Channel', 'Coupling Interval', 'Data', 'Label 1', 'Label 2', 'Patient', 'Type', 'S1/S2']]


x_test = X_test_features.drop(['Channel', 'Coupling Interval', 'Data', 'Label 1', 'Label 2', 'Patient', 'Type', 'S1/S2'], axis=1)
y_test = X_test_features['Label 1']
info_test = X_test_features[['Channel', 'Coupling Interval', 'Data', 'Label 1', 'Label 2', 'Patient', 'Type', 'S1/S2']]

In [59]:
def print_cm(cm, labels, hide_zeroes=False, hide_diagonal=False, hide_threshold=None):
    """pretty print for confusion matrixes"""
    columnwidth = max([len(x) for x in labels] + [5])  # 5 is value length
    empty_cell = " " * columnwidth
    
    # Begin CHANGES
    fst_empty_cell = (columnwidth-3)//2 * " " + "t/p" + (columnwidth-3)//2 * " "
    
    if len(fst_empty_cell) < len(empty_cell):
        fst_empty_cell = " " * (len(empty_cell) - len(fst_empty_cell)) + fst_empty_cell
    # Print header
    print("    " + fst_empty_cell, end=" ")
    # End CHANGES
    
    for label in labels:
        print("%{0}s".format(columnwidth) % label, end=" ")
        
    print()
    # Print rows
    for i, label1 in enumerate(labels):
        print("    %{0}s".format(columnwidth) % label1, end=" ")
        for j in range(len(labels)):
            cell = "%{0}.1f".format(columnwidth) % cm[i, j]
            if hide_zeroes:
                cell = cell if float(cm[i, j]) != 0 else empty_cell
            if hide_diagonal:
                cell = cell if i != j else empty_cell
            if hide_threshold:
                cell = cell if cm[i, j] > hide_threshold else empty_cell
            print(cell, end=" ")
        print()

## Train Channel Specific Model Using All Features

In [60]:
# Get cross validation scores on training data, following by test score.
import warnings
warnings.filterwarnings('ignore')

models = (LogisticRegression(penalty='l2', C=0.1, random_state=0, solver='liblinear', class_weight="balanced"),  
          svm.LinearSVC(penalty='l2', C=0.1, dual=False, class_weight="balanced"),
          naive_bayes.GaussianNB(),  
          gaussian_process.GaussianProcessClassifier(kernel=1.0*RBF(1)), 
          xgboost.XGBClassifier(),
          RandomForestClassifier())
model_names = ('Logistic Regression', 'Linear SVC', 'Naive Bayes', 'GP', 'XGBoost', 'Random Forest')

print('CS1-2 Model')
for clf, model_name in zip(models, model_names):
    print('~~~~~~~~~~~~~~~~~~~~~~~~~')
    print(model_name)
    print('Cross validation score:')
    print(cross_val_score(clf, x_training_cs12, y_training_cs12.values, cv=3))
    

print('\n\nCS3-4 Model')    
for clf, model_name in zip(models, model_names):
    print('~~~~~~~~~~~~~~~~~~~~~~~~~')
    print(model_name)
    print('Cross validation score:')
    print(cross_val_score(clf, x_training_cs34, y_training_cs34.values, cv=3))
    
print('\n\nCS5-6 Model') 
for clf, model_name in zip(models, model_names):
    print('~~~~~~~~~~~~~~~~~~~~~~~~~')
    print(model_name)
    print('Cross validation score:')
    print(cross_val_score(clf, x_training_cs56, y_training_cs56.values, cv=3))

CS1-2 Model
~~~~~~~~~~~~~~~~~~~~~~~~~
Logistic Regression
Cross validation score:
[0.94594595 0.59722222 0.88888889]
~~~~~~~~~~~~~~~~~~~~~~~~~
Linear SVC
Cross validation score:
[0.94594595 0.75       0.875     ]
~~~~~~~~~~~~~~~~~~~~~~~~~
Naive Bayes
Cross validation score:
[0.94594595 0.83333333 0.88888889]
~~~~~~~~~~~~~~~~~~~~~~~~~
GP
Cross validation score:
[0.91891892 0.93055556 0.90277778]
~~~~~~~~~~~~~~~~~~~~~~~~~
XGBoost
Cross validation score:
[0.91891892 0.81944444 0.93055556]
~~~~~~~~~~~~~~~~~~~~~~~~~
Random Forest
Cross validation score:
[0.91891892 0.86111111 0.93055556]


CS3-4 Model
~~~~~~~~~~~~~~~~~~~~~~~~~
Logistic Regression
Cross validation score:
[0.67567568 0.70833333 0.75      ]
~~~~~~~~~~~~~~~~~~~~~~~~~
Linear SVC
Cross validation score:
[0.75675676 0.77777778 0.81944444]
~~~~~~~~~~~~~~~~~~~~~~~~~
Naive Bayes
Cross validation score:
[0.93243243 0.84722222 0.95833333]
~~~~~~~~~~~~~~~~~~~~~~~~~
GP
Cross validation score:
[0.93243243 0.94444444 0.94444444]
~~~~~~~~~~

## Results of Classifiers - Feature Selection Using Feature Importance
Here features are selected using feature importance/recursive feature elimination. 

In [61]:
# # Get cross validation scores on training data, following by test score.
# import warnings
# warnings.filterwarnings('ignore')

# models = (LogisticRegression(penalty='l2', C=1, random_state=0, solver='liblinear', class_weight="balanced"), 
#           svm.SVC(class_weight="balanced"), 
#           svm.LinearSVC(penalty='l2', C=1, dual=False, class_weight="balanced"),
#           naive_bayes.GaussianNB(),  
#           gaussian_process.GaussianProcessClassifier(kernel=1.0*RBF(1)), 
#           xgboost.XGBClassifier(scale_pos_weight=(1/prop_fractionated)))
# model_names = ('Logistic Regression', 'RBF SVC', 'Linear SVC', 'Naive Bayes', 'GP', 'XGBoost')

# feature_names = x_training_cs12.columns

# print('Original number of features: ' + str(x_training_cs12.shape[1]))

# # Select according to feature importance. 
# # xgb = xgboost.XGBClassifier(scale_pos_weight=(1/prop_fractionated))
# # linear_svc = svm.LinearSVC(penalty='l1', C=10, dual=False)
# # log_regression = LogisticRegression(penalty='l1', C=10, random_state=0, solver='liblinear')


# # sfm = RFECV(log_regression, step=1, cv=3)
# # sfm.fit(x_training_cs12.values, y_training_cs12.values)
# # x_training_sparse = sfm.transform(x_training_cs12.values)
# x_training_sparse = x_training_cs12.values
# print('Number of features selected: ' + str(x_training_sparse.shape[1]))
# mask = sfm.get_support() #list of booleans
# selected_features = [] # The list of your K best features

# for bool, feature in zip(mask, feature_names):
#     if bool:
#         selected_features.append(feature)
        
# print(selected_features)

# for clf, model_name in zip(models, model_names):
#     print('~~~~~~~~~~~~~~~~~~~~~~~~~')
#     print(model_name)
#     print('Cross validation score:')
#     print(cross_val_score(clf, x_training_sparse, y_training_cs12.values, cv=3))

# # print('Test Score:')
# # X_test_sparse = sfm.transform(X_test.values)
# # xgb.fit(X_training_sparse, y_training_cs12.values)
# # print(xgb.score(X_test_sparse, y_test.values))

In [62]:
# x_test_sparse = sfm.transform(x_test_cs12.values)

print('CS1-2 Model')
for clf, model_name in zip(models, model_names):
    print('~~~~~~~~~~~~~~~~~~~~~~~~~')
    print(model_name)
    clf.fit(x_training_cs12.values, y_training_cs12.values)
    print('Test data score:')
    print(clf.score(x_test_cs12.values, y_test_cs12.values))
    predictions = clf.predict(x_test_cs12.values)
    cm = confusion_matrix(y_test_cs12.values, predictions)
    print_cm(cm, ['Not Fractionated','Fractionated'])
    

print('\n\nCS3-4 Model')    
for clf, model_name in zip(models, model_names):
    print('~~~~~~~~~~~~~~~~~~~~~~~~~')
    print(model_name)
    clf.fit(x_training_cs34.values, y_training_cs34.values)
    print('Test data score:')
    print(clf.score(x_test_cs34.values, y_test_cs34.values))
    predictions = clf.predict(x_test_cs34.values)
    cm = confusion_matrix(y_test_cs34.values, predictions)
    print_cm(cm, ['Not Fractionated','Fractionated'])
    
print('\n\nCS5-6 Model') 
for clf, model_name in zip(models, model_names):
    print('~~~~~~~~~~~~~~~~~~~~~~~~~')
    print(model_name)
    clf.fit(x_training_cs56.values, y_training_cs56.values)
    print('Test data score:')
    print(clf.score(x_test_cs56.values, y_test_cs56.values))
    predictions = clf.predict(x_test_cs56.values)
    cm = confusion_matrix(y_test_cs56.values, predictions)
    print_cm(cm, ['Not Fractionated','Fractionated'])

CS1-2 Model
~~~~~~~~~~~~~~~~~~~~~~~~~
Logistic Regression
Test data score:
0.819047619047619
           t/p       Not Fractionated     Fractionated 
    Not Fractionated             69.0             15.0 
        Fractionated              4.0             17.0 
~~~~~~~~~~~~~~~~~~~~~~~~~
Linear SVC
Test data score:
0.8761904761904762
           t/p       Not Fractionated     Fractionated 
    Not Fractionated             76.0              8.0 
        Fractionated              5.0             16.0 
~~~~~~~~~~~~~~~~~~~~~~~~~
Naive Bayes
Test data score:
0.9047619047619048
           t/p       Not Fractionated     Fractionated 
    Not Fractionated             81.0              3.0 
        Fractionated              7.0             14.0 
~~~~~~~~~~~~~~~~~~~~~~~~~
GP
Test data score:
0.8666666666666667
           t/p       Not Fractionated     Fractionated 
    Not Fractionated             84.0              0.0 
        Fractionated             14.0              7.0 
~~~~~~~~~~~~~~~~~~~~~~~

In [45]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

log_reg = LogisticRegression(penalty='l2', C=0.1, random_state=0, solver='liblinear', class_weight="balanced")
xgb = xgboost.XGBClassifier()
rf = RandomForestClassifier()
feature_names = x_training_cs12.columns

sfs1 = SFS(rf, 
           k_features=(4,8), 
           forward=True, 
           floating=False, 
           scoring='roc_auc',
           cv=3,
           verbose=2)

y_training_cs12_bi = [int(y) for y in y_training_cs12.values]
sfs1 = sfs1.fit(x_training_cs12.values, y_training_cs12_bi, custom_feature_names=feature_names)
print('Selected features:', sfs1.k_feature_names_)

x_training_cs12_sfs = sfs1.transform(x_training_cs12.values)
x_test_cs12_sfs = sfs1.transform(x_test_cs12.values)

rf.fit(x_training_cs12_sfs, y_training_cs12.values)
print(rf.score(x_test_cs12_sfs, y_test_cs12.values))

predictions = rf.predict(x_test_cs12_sfs)
cm = confusion_matrix(y_test_cs12.values, predictions)
print_cm(cm, ['Not Fractionated','Fractionated'])

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.4s finished

[2019-03-03 22:29:19] Features: 1/8 -- score: 0.8975904789776609[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.4s finished

[2019-03-03 22:29:19] Features: 2/8 -- score: 0.911248902546093[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.4s finished

[2019-03-03 22:29:20] Features: 3/8 -- score: 0.9281216466686177[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.3s finished

[2019-03-03 22:29:20] Features: 4/8 -- score: 0.9220002926543751[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.2s 

Selected features: ('Approximate Entropy: m=3 r=0.7', 'Location of Maximum Energy: M=14', 'Number of Peaks: set_thresh=False', 'Width of Maximum Energy: M=14, width_thresh=0.2')
0.9142857142857143
           t/p       Not Fractionated     Fractionated 
    Not Fractionated             84.0              0.0 
        Fractionated              9.0             12.0 


In [38]:
sfs1 = SFS(rf, 
           k_features=(4,8), 
           forward=True, 
           floating=False, 
           scoring='roc_auc',
           cv=3, 
           verbose=2)

y_training_cs34_bi = [int(y) for y in y_training_cs34.values]
sfs1 = sfs1.fit(x_training_cs34.values, y_training_cs34_bi, custom_feature_names=feature_names)
print('Selected features:', sfs1.k_feature_names_)

x_training_cs34_sfs = sfs1.transform(x_training_cs34.values)
x_test_cs34_sfs = sfs1.transform(x_test_cs34.values)

rf.fit(x_training_cs34_sfs, y_training_cs34.values)
print(rf.score(x_test_cs34_sfs, y_test_cs34.values))

predictions = rf.predict(x_test_cs34_sfs)
cm = confusion_matrix(y_test_cs34.values, predictions)
print_cm(cm, ['Not Fractionated','Fractionated'])

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.4s finished

[2019-03-03 22:27:37] Features: 1/8 -- score: 0.8350099460073884[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.4s finished

[2019-03-03 22:27:37] Features: 2/8 -- score: 0.8791542341574311[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.4s finished

[2019-03-03 22:27:38] Features: 3/8 -- score: 0.9106741972151179[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.4s finished

[2019-03-03 22:27:38] Features: 4/8 -- score: 0.9041435777209434[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.3s

Selected features: ('Approximate Entropy: m=3 r=0.7', 'Index Mass Quantile: q=0.6', 'Location of Maximum Energy: M=14', 'Percentage Fractionation: thresh=0.01', 'Width of Maximum Energy: M=14, width_thresh=0.2')
0.8285714285714286
           t/p       Not Fractionated     Fractionated 
    Not Fractionated             76.0             16.0 
        Fractionated              2.0             11.0 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s finished

[2019-03-03 22:27:39] Features: 8/8 -- score: 0.8630665672065927

In [48]:
sfs1 = SFS(rf, 
           k_features=(4,8), 
           forward=True, 
           floating=False, 
           scoring='roc_auc',
           cv=3,
           verbose=2)

y_training_cs56_bi = [int(y) for y in y_training_cs56.values]
sfs1 = sfs1.fit(x_training_cs56.values, y_training_cs56_bi, custom_feature_names=feature_names)
print('Selected features:', sfs1.k_feature_names_)

x_training_cs56_sfs = sfs1.transform(x_training_cs56.values)
x_test_cs56_sfs = sfs1.transform(x_test_cs56.values)

rf.fit(x_training_cs56_sfs, y_training_cs56.values)
rf.score(x_test_cs56_sfs, y_test_cs56.values)

predictions = rf.predict(x_test_cs56_sfs)
cm = confusion_matrix(y_test_cs56.values, predictions)
print_cm(cm, ['Not Fractionated','Fractionated'])

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.4s finished

[2019-03-03 22:30:26] Features: 1/8 -- score: 0.9751068376068376[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.4s finished

[2019-03-03 22:30:26] Features: 2/8 -- score: 0.9472360972360973[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.4s finished

[2019-03-03 22:30:27] Features: 3/8 -- score: 0.9868520368520368[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.4s finished

[2019-03-03 22:30:27] Features: 4/8 -- score: 0.9817238317238317[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.2s

Selected features: ('Approximate Entropy: m=3 r=0.7', 'Index Mass Quantile: q=0.6', 'Number of Peaks: set_thresh=False', 'Sample Entropy Around Max Energy: width=60 r=0.025', 'Width of Maximum Energy: M=14, width_thresh=0.2')
           t/p       Not Fractionated     Fractionated 
    Not Fractionated             96.0              3.0 
        Fractionated              1.0              5.0 


In [4]:
def get_good_feature_dict(x, col_prefix=''):
    feature_dict = {}
    height_thresh=0.1
    
    feature_dict[col_prefix + 'Maximum Absolute Value'] = max(abs(x))
    
    # Hand engineered features
    x = x/max(abs(x))
#     feature_dict[col_prefix + 'Conduction Delay: set_thresh=False'] = get_delay(x)
    peaks = get_peaks(x, height_thresh)
    feature_dict[col_prefix + 'Number of Peaks: set_thresh=False'] = len(peaks[0])
    feature_dict[col_prefix + 'Percentage Fractionation: thresh=0.01'] = percentage_fractionation(x, peaks[0], thresh=0.01)
    
    # Denoise x for remaining features
    x = denoise(x)
    max_energy_idx = get_location_of_max_energy(x)
    feature_dict[col_prefix + 'Location of Maximum Energy: M=14'] = max_energy_idx
    feature_dict[col_prefix + 'Sample Entropy Around Max Energy: width=60 r=0.025'] = get_local_sample_entropy(x, max_energy_idx, 60, m=2, r=0.025)
    feature_dict[col_prefix + 'Width of Maximum Energy: M=14, width_thresh=0.2'] = get_width_max_energy(x, M=14, width_thresh=0.2)
    
    # Temporal features
    feature_dict[col_prefix + 'Approximate Entropy: m=3 r=0.7'] = feature_calculators.approximate_entropy(x, 3, 0.7)
    imq = feature_calculators.index_mass_quantile(x, [{'q': 0.6}])
    feature_dict[col_prefix + 'Index Mass Quantile: q=0.6'] = imq[0][1]
    feature_dict[col_prefix + 'Ratio Beyond 1xSTD'] = feature_calculators.ratio_beyond_r_sigma(x, 1)
    
    # Spectral features
#     feature_dict[col_prefix + 'Power Spectral Entropy'] = get_pse(x)
    
    return feature_dict
    

def get_hand_engineered_feature_dict(x, thresh_cd=None, set_thresh_cd=False, thresh_peaks=None, set_thresh_peaks=False, show_peaks=False, col_prefix = ''):
    feature_dict = {}
    sf = max(abs(x))
    x = x/max(abs(x))

    # Hand engineered features
    if set_thresh_cd:
        thresh_cd = thresh_cd/sf
        feature_dict[col_prefix + 'Conduction Delay: set_thresh=True'] = get_delay(x, thresh_cd, set_thresh_cd)
        feature_dict[col_prefix + 'Conduction Delay: set_thresh=False'] = get_delay(x)
    else:
        feature_dict[col_prefix + 'Conduction Delay: set_thresh=False'] = get_delay(x)
    
    height_thresh=0.1
    if set_thresh_peaks:
        thresh_peaks = thresh_peaks/sf
        peaks = get_peaks(x, height_thresh, thresh_peaks, set_thresh_peaks, plot=False)
        feature_dict[col_prefix + 'Number of Peaks: set_thresh=True'] = len(peaks[0])
        peaks = get_peaks(x, height_thresh)
        feature_dict[col_prefix + 'Number of Peaks: set_thresh=False'] = len(peaks[0])
    else:
        peaks = get_peaks(x, height_thresh)
        feature_dict[col_prefix + 'Number of Peaks: set_thresh=False'] = len(peaks[0])
    
    peaks = get_peaks(x, height_thresh)
    feature_dict[col_prefix + 'Percentage Fractionation: thresh=0.01'] = percentage_fractionation(x, peaks[0], thresh=0.01)
    
    # Denoise x for remaining features
    x = denoise(x)
    
    max_energy_idx = get_location_of_max_energy(x)
    feature_dict[col_prefix + 'Location of Maximum Energy: M=14'] = max_energy_idx
    feature_dict[col_prefix + 'Sample Entropy Around Max Energy: width=60 r=0.025'] = get_local_sample_entropy(x, max_energy_idx, 60, m=2, r=0.025)
    feature_dict[col_prefix + 'Energy Around Max Energy'] = get_local_energy(x, max_energy_idx, 60)
    min_idx = np.argmin(x)
    max_idx = np.argmax(x)
    feature_dict[col_prefix + 'Peaks Between Min and Max'] = len([i for i in peaks[0] if ((i > min_idx) & (i < max_idx))])
    feature_dict[col_prefix + 'Width of Maximum Energy: M=14, width_thresh=0.4'] = get_width_max_energy(x, M=14, width_thresh=0.4)
    feature_dict[col_prefix + 'Width of Maximum Energy: M=14, width_thresh=0.2'] = get_width_max_energy(x, M=14, width_thresh=0.2)

    return feature_dict

def get_spectral_feature_dict(x, col_prefix = ''):
    feature_dict = {}
    # Denoise and normalise x for remaining features
    x = denoise(x)
    x = x/max(abs(x))
    
    feature_dict[col_prefix + 'Power Spectral Entropy'] = get_pse(x)
    feature_dict[col_prefix + 'Spectral Centroid'] = get_spectral_centroid(x)
    max_energy_idx = get_location_of_max_energy(x)
    feature_dict[col_prefix + 'Power Spectral Entropy Around Maximum Energy: width=30'] = get_local_pse(x, max_energy_idx, width=30)
    feature_dict[col_prefix + 'Spectral Centroid Around Maximum Energy: width=30'] = get_local_spectral_centroid(x, max_energy_idx, width=30)
    feature_dict[col_prefix + 'Power Spectral Entropy Around Maximum Energy: width=60'] = get_local_pse(x, max_energy_idx, width=60)
    feature_dict[col_prefix + 'Spectral Centroid Around Maximum Energy: width=60'] = get_local_spectral_centroid(x, max_energy_idx, width=60)
    
    return feature_dict
    
def get_temporal_feature_dict(x, col_prefix = ''):

    feature_dict = {}
    feature_dict[col_prefix + 'Maximum Absolute Value'] = np.max(abs(x))
    
    # Denoise and normalise x for remaining features
    x = denoise(x)
    x = x/max(abs(x))


    erbc = feature_calculators.energy_ratio_by_chunks(x, [{'num_segments':10, 'segment_focus':3}, {'num_segments':10, 'segment_focus':2}])
    feature_dict[col_prefix + 'Energy Ratio by Chunks: num_segments=10 segment_focus=2'] = erbc[1][1]
    feature_dict[col_prefix + 'Energy Ratio by Chunks: num_segments=10 segment_focus=3'] = erbc[0][1]
    feature_dict[col_prefix + 'Approximate Entropy: m=3 r=0.7'] = feature_calculators.approximate_entropy(x, 3, 0.7)
    feature_dict[col_prefix + 'Ratio Beyond 5xSTD'] = feature_calculators.ratio_beyond_r_sigma(x, 5)
    feature_dict[col_prefix + 'Ratio Beyond 4xSTD'] = feature_calculators.ratio_beyond_r_sigma(x, 4)
    feature_dict[col_prefix + 'Ratio Beyond 3xSTD'] = feature_calculators.ratio_beyond_r_sigma(x, 3)
    feature_dict[col_prefix + 'Ratio Beyond 2xSTD'] = feature_calculators.ratio_beyond_r_sigma(x, 2)
    feature_dict[col_prefix + 'Ratio Beyond 1xSTD'] = feature_calculators.ratio_beyond_r_sigma(x, 1)
    # A fraction q of the mass lies to the left of i. (Alternative to conduction delay?)
    imq = feature_calculators.index_mass_quantile(x, [{'q': 0.6}, {'q': 0.4}])
    feature_dict[col_prefix + 'Index Mass Quantile: q=0.6'] = imq[0][1]
    feature_dict[col_prefix + 'Index Mass Quantile: q=0.4'] = imq[1][1]
    

    return feature_dict

In [5]:
# A shitty conduction delay detector
def get_delay(x, amp_thresh=None, set_thresh=False):
    if (set_thresh==True):
        if any(abs(x)>amp_thresh):
            return np.argmax(abs(x)>amp_thresh)
        else:
            return len(x)
    else:    
        return np.argmax(abs(x)>(max(abs(x))/2))
    
def denoise(x):
    # Obtain Daubechies N=6 wavelet coefficients
    waveletCoefs = pywt.wavedec(x, 'db7', mode='per')

    # Throw away coefficients corresponding to noise
    sigma = mad(waveletCoefs[-1])
    uThresh = 1*sigma*np.sqrt(2*np.log(len(x)))
    denoised = waveletCoefs[:]
    denoised[1:] = (pywt._thresholding.hard(i, value=uThresh) for i in denoised[1:])

    # Reconstruct the original signal
    xDenoised = pywt.waverec(denoised, 'db7', mode='per')

    return xDenoised

def get_peaks(x, height_thresh, scale_amp=None, set_scale=False, plot = False):
    x = np.array(x)
    
    # Get height_thresh
    if set_scale:
        height_thresh = height_thresh*scale_amp
    else:
        height_thresh = height_thresh*max(abs(x))
    
    # Denoise x
    xdn = denoise(x)

    # Detect peaks using detect_peaks
    pos_peak_idx = detect_peaks(xdn, mph=height_thresh, threshold = 0)
    neg_peak_idx = detect_peaks((-xdn), mph=height_thresh, threshold = 0)
    peak_idx = np.concatenate([pos_peak_idx, neg_peak_idx])
    peak_idx = np.sort(peak_idx)
    # Edge indeces aren't detected
    peak_idx = peak_idx[(peak_idx != 0) & (peak_idx != (len(xdn)-1))]

    new_peak_idx = []
    peak_amp = []
    if (len(peak_idx) > 0):
        new_peak_idx.append(peak_idx[0])
        mp_thresh = 0.2*max(abs(x))
        for i in range(len(peak_idx)-1):
            idx = peak_idx[i]
            idx_next = peak_idx[i+1]
            mid_point = int((idx_next+idx)/2)
            if (max([abs(x[idx_next]-x[mid_point]), abs(x[idx]-x[mid_point])]) > mp_thresh):
                new_peak_idx.append(idx_next)

        peak_idx = np.array(new_peak_idx)
        peak_amp = x[peak_idx]

    if plot == True:
        fig, [ax1] = plt.subplots(nrows=1, ncols=1, sharex=True, figsize=(8,8))
        ax1.plot(x, 'b' , xdn, 'r--', peak_idx, peak_amp, 'kx')
        #plt.title(fileName)
        ax1.set_xlabel('Sample')
        ax1.set_ylabel('Normalised amplitude')
        ax1.legend(['Original segment', 'Denoised segment', 'Detected peaks'])

        plt.draw()
        plt.waitforbuttonpress(0) # this will wait for indefinite time
        plt.close(fig)


    return peak_idx, peak_amp

def sample_entropy(U, m, r):

    def _maxdist(x_i, x_j):
        result = max([abs(ua-va) for ua, va in zip(x_i, x_j)])
        return result

    def _phi(m):
        x = np.zeros([N,m-1])
        for i in range(N-m+1):
            x[i,:] = U[i:i+m-1]

        C = 0
        for i in range(len(x)):
            for j in range(len(x)):
                if i != j:
                    if _maxdist(x[i,:], x[j,:]) <= r:
                        C = C + 1

        return C

    U = U/max(abs(U))
    N = len(U)

    return -np.log(_phi(m+1)/_phi(m))

def percentage_fractionation(x, peak_idxs, thresh=0.01, sr=1000):
    # Get peak indexes and amplitude
    peak_idx_diffs = np.diff(peak_idxs)
    frac_time = 0
    frac_time = np.sum(peak_idx_diffs[peak_idx_diffs < thresh*sr])
    prcnt_frac = (frac_time/len(x))*100
    return prcnt_frac

def get_local_sample_entropy(x, centre_idx, width, m=2, r=0.05):
    # Ensure width is odd
    if ((width%2) == 0):
        width += 1
        
    if (centre_idx < (width-1)/2):
        return sample_entropy(x[:width+1], m, r)
    elif (centre_idx > (len(x)-1-(width-1)/2)):
        return sample_entropy(x[len(x)-1-width:], m, r)
    else:
        return sample_entropy(x[int(centre_idx-(width-1)/2):int(centre_idx+(width+1)/2)], m, r)
    
def get_location_of_max_energy(x, M=14):
    v = np.ones(M)
    x_ = np.convolve(abs(x), v)
    return (np.argmax(x_) + math.floor(M/2))
        
def get_local_peaks(x, centre_idx, width=25, height_thresh=0.1):
    if ((width%2) == 0):
        width += 1
        
    if (centre_idx < (width-1)/2):
        return get_peaks(x[:width+1], height_thresh)
    elif (centre_idx > (len(x)-1-(width-1)/2)):
        return get_peaks(x[len(x)-1-width:], height_thresh)
    else:
        return get_peaks(x[int(centre_idx-(width-1)/2):int(centre_idx+(width+1)/2)], height_thresh)
    
def get_pse(x):
    x_fft = np.fft.rfft(x)
    x_P = (1/len(x_fft))*np.absolute(x_fft)**2
    x_p = x_P/sum(x_P)
    pse = np.sum([(-p*np.log2(p)) for p in x_p])
    return pse

def get_local_pse(x, centre_idx, width=50):
    if ((width%2) == 0):
        width += 1
        
    if (centre_idx < (width-1)/2):
        return get_pse(x[:width+1])
    elif (centre_idx > (len(x)-1-(width-1)/2)):
        return get_pse(x[len(x)-1-width:])
    else:
        return get_pse(x[int(centre_idx-(width-1)/2):int(centre_idx+(width+1)/2)])
    
def get_spectral_centroid(x):
    x_fft = np.fft.rfft(x)
    x_spectrum = np.absolute(x_fft)
    normalized_spectrum = x_spectrum/sum(x_spectrum)
    normalized_frequencies = np.arange(0, len(x_spectrum), 1)
    return sum(normalized_frequencies * normalized_spectrum)

def get_local_spectral_centroid(x, centre_idx, width=50):
    if ((width%2) == 0):
        width += 1
        
    if (centre_idx < (width-1)/2):
        return get_spectral_centroid(x[:width+1])
    elif (centre_idx > (len(x)-1-(width-1)/2)):
        return get_spectral_centroid(x[len(x)-1-width:])
    else:
        return get_spectral_centroid(x[int(centre_idx-(width-1)/2):int(centre_idx+(width+1)/2)])
    
def get_local_energy(x, centre_idx, width=60):
    if ((width%2) == 0):
        width += 1
        
    if (centre_idx < (width-1)/2):
        return np.sum(x[:width+1]**2)
    elif (centre_idx > (len(x)-1-(width-1)/2)):
        return np.sum(x[len(x)-1-width:]**2)
    else:
        return np.sum(x[int(centre_idx-(width-1)/2):int(centre_idx+(width+1)/2)]**2)
    
def get_width_max_energy(x, M=14, width_thresh=0.2):
    v = np.ones(M)
    x_ = np.convolve(abs(x), v)
    if any(x_[np.argmax(x_):] < width_thresh*np.max(x_)):
        end_idx = np.argmax(x_) + np.argmax(x_[np.argmax(x_):] < width_thresh*np.max(x_))
    else:
        end_idx = len(x_)-1
    if any(x_[np.argmax(x_)::-1] < width_thresh*np.max(x_)):  
        start_idx = np.argmax(x_) - np.argmax(x_[np.argmax(x_)::-1] < width_thresh*np.max(x_))
    else:
        start_idx = 0
    return (end_idx - start_idx)